In [3]:
import pandas as pd
df = pd.read_csv('countries.csv')
df = df[df['country'].notna()]#remove any NAN values
data = df.to_dict('records')

In [4]:
from qdrant_client import models,QdrantClient
from sentence_transformers import SentenceTransformer

c:\Users\user\Desktop\Coursera-projects\Rag-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
encoder = SentenceTransformer('all-MiniLm-L6-v2')#model to create embeddings

In [6]:
qdrant = QdrantClient(":memory:") #create in-memory Qdrant instance

In [7]:
#create collection to store countries
qdrant.recreate_collection(
    collection_name="countries",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),#vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\user\AppData\Local\Temp\ipykernel_4996\2073917227.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [8]:
#vectorize
qdrant.upload_points(
    collection_name="countries",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["description"]).tolist(),
            payload=doc,
        )for idx , doc in enumerate(data)#data is the variable holding all the countries
        
    ]
)

c:\Users\user\Desktop\Coursera-projects\Rag-project\.venv\Lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [13]:
user_prompt=input("Enter a country name or description: ")

In [ ]:
hits =qdrant.search(
    collection_name="countries",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
    print(hit.payload)

In [11]:
search_results = [hit.payload for hit in hits]

In [14]:
#now time to connect to the local large language model
from openai import OpenAI

client = OpenAI(
    base_url="http://192.168.1.8:8080/v1",
    api_key="sk-no-key-required"
)

completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role":"system","content":"You are a helpful geography assistant. When the user gives you the name of a country, respond with useful and interesting information about that country. Do not recommend other countries."},
        {"role":"user","content":user_prompt}
    ],
    max_tokens=100,
    temperature=0.7,
    top_p=0.9,
)

print(completion.choices[0].message.content)


The capital of France is Paris. Paris is known as the City of Light and is famous for its art, culture, and cuisine. The city is home to many famous landmarks, including the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also known for its romantic atmosphere and is a popular destination for tourists from around the world.
